In [ ]:
# Source the package setup script
source("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/scripts/00_setup_packages.R")

# Source the custom graphing functions
source("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/scripts/01_graphing_functions.R")


# Model 6

## Question

Does camouflage strength (**edge disruption**) vary between males and females across microhabitats, and by acuity?

## Objective

Test for the effect of **sex**, **microhabitat**, and **acuity** on **edge disruption**. 

## Method

### 1. Load cleaned data.

We start by loading the cleaned data from the "03_data_cleaning" pipeline. This data has already undergone transformations and contains relevant metrics for our models.


In [ ]:

data_m6_clean <- read.csv("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/cleaned/data_m6_clean.csv")

---

### 2. Prepare data for modeling.

#### **Categorical predictors**

For categorical and binary predictors, R automatically dummy-codes the variables. For binary variables, such as Sex (0 = Female, 1 = Male), R sets the baseline to 0 unless specified otherwise. This ensures comparisons are made relative to the baseline group.

#### **Continuous predictors**

We standardize continuous predictors, such as Fecundity, by centering and scaling them (dividing by two standard deviations). This improves interpretability, aligning their coefficients with those of binary predictors, as suggested by Gelman (2008).


In [ ]:
# Convert categorical variables to factors
columns_to_convert_m6 <- c("Image", "Morph", "Sex", "Microhabitat_Association", "Viewpoint", "Microhabitat", "Background")

data_m6_clean <- data_m6_clean %>%
    mutate(across(all_of(columns_to_convert_m6), as.factor))

# Set reference category for categorical predictors
data_m6_clean$Sex <- relevel(data_m6_clean$Sex, ref = "M")
data_m6_clean$Microhabitat <- relevel(data_m6_clean$Microhabitat, ref = "Hydroid")
data_m6_clean$Viewpoint <- relevel(data_m6_clean$Viewpoint, ref = "0")



# Convert continuous variables to numeric
columns_to_convert_m6 <- c("GabRat")

# Convert columns
data_m6_clean <- data_m6_clean %>%
    mutate(across(all_of(columns_to_convert_m6), as.numeric))


---

### 3. Visualize response variable distributions

To understand the distributions of our response variables, we plot density curves. This helps confirm whether a Beta distribution is appropriate for modeling, as Beta is suited for positively skewed, continuous data.


In [ ]:

p_response_m6_GabRat <- data_m6_clean %>%
    ggplot(aes(x = GabRat)) +
    geom_density(fill = "#69b3a2", color = "#e9ecef", alpha = 0.8) +
    theme_bw() +
    labs(title = "GabRat") +
    theme(axis.title.x = element_blank())



# Combine plots for easy visualization
plot_responses_m6 <- ggarrange(
    p_response_m6_GabRat,
    ncol = 1,
    nrow = 1
)

annotate_figure(
    plot_responses_m6,
    top = text_grob("Ranges of response variables")
)

ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_responses_m6.png", plot = plot_responses_m6, width = 3, height = 3, units = "in", dpi = 300)


In [ ]:
# Convert images to base64
plot_responses_m6 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_responses_m6.png")

# Create the HTML (vertical display)
html_edge_disruption <- paste0("
  <style>
    .image-container {
      display: flex;
      justify-content: space-between;
      align-items: flex-start;
      flex-wrap: wrap;
      gap: 20px;
      max-width: 100%;
    }

    .image-container img {
      flex: 1 1 48%;
      max-width: 48%;
      height: auto;
      border: 1px solid #ccc;
    }

    @media screen and (max-width: 800px) {
      .image-container img {
        max-width: 100%;
        flex: 1 1 100%;
      }
    }
  </style>

  <div class='image-container'>
    <img src='", plot_responses_m6, "' alt='Edge Disruption'>
  </div>
")

# Display the HTML
IRdisplay::display_html(html_edge_disruption)


---

### 4. Justification for GLMs and Bayesian methods

#### **Why GLMs?**

Generalized Linear Models (GLMs) are used because our response variable is non-negative and constrained between 0 and 1. Linear models assume an unbounded, continuous range of possible outcomes. The Beta distribution with a logit link function allows us to model these variables appropriately.

---

#### **Why Bayesian methods?**

Bayesian GLMs were chosen over frequentist approaches because:

-   **Sparse data**: Bayesian methods handle sparse datasets more robustly.

-   **Priors**: They allow us to incorporate prior knowledge, improving model performance.

-   **Posterior distributions**: Bayesian models provide posterior distributions, offering a full view of parameter uncertainty.

---

### 5. Define priors

#### **What are priors?**

In Bayesian statistics, priors represent our beliefs about parameter values before analyzing the data. These beliefs are mathematically expressed as probability distributions. Priors guide the model, especially when data are sparse or when the signal-to-noise ratio is low.

---

#### **Why priors matter:**

-   **Prevent overfitting**: Priors discourage extreme parameter estimates unless strongly supported by the data.

-   **Balance restrictiveness and flexibility**: Weakly informative priors let the data dominate while providing reasonable bounds.

-   **Leverage existing knowledge**: Informative priors incorporate previous research or domain expertise, improving accuracy in well-studied systems.

---

#### **How priors work in this analysis:**
We combine the priors (representing initial beliefs) with the likelihood of the observed data to compute posterior distributions, which reflect updated beliefs after observing the data.

The general formula is:

$$
\begin{aligned}
\text{Posterior } \alpha \text{ Likelihood} * \text{Prior}
\end{aligned}
$$

---

#### **Model family and formula**
The response variable (GabRat) is continuous and theoretically bounded between 0 and 1, making the Beta distribution suitable. To ensure the model captures this bounded nature effectively, a logit link function is applied, transforming $\mu_i$ (mean of the Beta distribution) into the real number line.

The Beta distribution is parameterized as:

$$
\begin{aligned}
y_{i} &\sim \text{Beta}(\mu_{i}\phi, (1-\mu_i)\phi) \\
\\
& \text{logit}(\mu_{i}) = \alpha + \beta x_{i} \\
\end{aligned}
$$

$$
\begin{aligned}
\text{Where:} \\
y_{i}: & \text{ Response variable for observation i} \\
\mu_{i}: & \text{ Mean of the Beta distribution for i} \\
\phi: & \text{ Precision parameter (controls dispersion)} \\
\alpha: & \text{ Intercept, representing the baseline value of } \text{logit}(\mu) \\
\beta: & \text{ Coefficient, representing the effect of } x_{i} \text{ on }\text{logit}(\mu)
\end{aligned}
$$

Because the logit link transforms the mean ($\mu$) to the log-odds scale, slope coefficients ($\beta$) can be interpreted in terms of odds ratios, even though the response variable is continuous.

---
#### **Chosen priors and rationale**

**Intercept ($\alpha$)**: The intercepts represent the mean of the response variable when all predictors are at their baseline values. Since GabRat is a continuous, bounded variable (0 to 1), we set a weakly informative prior based on the logit-transformed range of the response mean. This approach ensures that the intercept (1) reflects biologically plausible values for the response variable and (2) allows for moderate variation around the mean without promoting extreme values.

Because GabRat rarely exceeds 0.8, our prior is designed to constrain estimates within a realistic range, while still permitting variation. This helps stabilize parameter estimation and ensure that the model accurately captures the bimodal nature of the data.

$$
\begin{aligned}
\alpha \approx N(-1,1) \\
\end{aligned}
$$

This prior allows for moderate variability on the logit scale (95% of values between -3 and 1), corresponding to values ranging from 0.029 to 0.62 on the original scale:

$$
\begin{aligned}
\mu = \frac{e^{\text{logit}(\mu)}}{1 + e^{\text{logit}(\mu)}}
\end{aligned}
$$

For $\text{logit}(\mu)=-1.5$:
$$
\begin{aligned}
\mu = \frac{e^{-1}}{1 + e^{-1}} = \frac{0.368}{1+0.368} = 0.269
\end{aligned}
$$

For $\text{logit}(\mu)=-3$:
$$
\begin{aligned}
\mu = \frac{e^{-3}}{1 + e^{-3}} = \frac{0.0498}{1+0.0498} = 0.047
\end{aligned}
$$

For $\text{logit}(\mu)=1$:
$$
\begin{aligned}
\mu = \frac{e^{1}}{1 + e^{1}} = \frac{2.72}{1+2.72} = 0.73
\end{aligned}
$$

---

**Slope ($\beta$)**

The slope represents the change in the logit-transformed mean response ($\mu$) for a one-unit increase in the predictor. On the odds scale, the slope is interpreted as a multiplicative scaling factor:

$$
\begin{aligned}
\text{Odds}_{x+1} = \text{Odds}_{x} * e^{\beta}
\end{aligned}
$$

For example:

-   If $\beta = 0.1$, a one-unit increase in the predictor scales the Odds response by $e^{0.1} \approx 1.10$ (a 10% increase).

-   If $\beta = -0.1$, a one-unit increase scales the Odds response by $e^{-0.1} \approx 0.91$ (a 9% decrease).

Because we do not know the direction or magnitude of the predictor's effect, we use a weakly informative prior centered at 0. This allows for moderate variability in either direction without promoting extreme values:

$$
\begin{aligned}
\beta \approx N(0,0.5)
\end{aligned}
$$

This prior places 95% of values between approximately $-1$ and $1$ on the logit scale, corresponding to odds ratios ranging from $e^{-1} \approx 0.37$ to $e^{1} \approx 2.72$. In other words, a one-unit change in the predictor could plausibly scale the odds of the outcome between 0.37× and 2.72× the baseline, while discouraging unrealistically large effects unless strongly supported by the data.

---

**Random effects ($\sigma$)** 

Random effects account for variability between groups that we are not explicitly testing. In this model, Images will be the random effect. This is because we have repeated measures within Images (i.e., different viewpoints of the same Image). Measurements within the same Image are likely to be correlated because they share the same combination of background and subject, which can introduce non-independence. Including random effects in our model accounts for this. To capture moderate variability, we assign:

$$
\begin{aligned}
\approx N(0,0.5)
\end{aligned}
$$

This ensures flexibility without overfitting.

---

**Precision Parameter ($\phi$)**: In a Beta distribution, dispersion is controlled by the precision parameter ($\phi$), which determines how tightly the data is concentrated around the mean. The precision parameter takes positive real values, with larger values indicating greater concentration around the mean—hence higher precision and lower dispersion.

*It is important to distinguish this concept of dispersion from variability as described in Model 5. Here, dispersion refers to a property of the Beta distribution (a probability distribution for proportions, often with a logit link), whereas variability in the Gamma distribution (Model 5) refers to the observed variation in the data itself.*

In brms, the precision parameter $\phi$ can be modeled as a constant or as a function of predictors. Instead of directly estimating $\phi$, the model estimates its log-transformed value via regression coefficients:

$$
\begin{aligned}
\text{log}(\phi) = \beta_0 + \beta_1X_1 + \beta_2X_2...
\end{aligned}
$$

The prior is applied to this log-transformed scale, allowing us to use normal distributions that include negative values.

We set the prior on $\log(\phi)$ to a weakly informative normal distribution $N(0, 1)$. This prior assumes that $\phi$ is likely to be around $e^0 = 1$, with most values ranging from $e^{-2} = 0.14$ to $e^{2} = 7.39$. This allows for flexible dispersion in the Beta distribution while discouraging extreme values.

NOTE: Your response will always be between 0 and 1 in a Beta model. The phi value just controls relative dispersion within this range.


---

#### **Visualizing priors**

To validate these priors, we run models sampling only from the priors (sample_prior = "only") and inspect their outputs to ensure they align with our expectations.


>**Note:** We will run these models in RStudio to be consistent because the rstan package sometimes does not like Jupyter. The models were saved in RStudio and loaded below. We left the code chunks as comments for reference.

In [ ]:

# # Set seed for reproducibility
# set.seed(5678)
# # Build model
# m6_priors_GabRat <- brm(
#     bf(GabRat ~ 1 + Viewpoint + Sex + Microhabitat + (1 | Image)),
#     data = data_m6_clean,
#     family = Beta(),
#     prior = c(
#         prior(normal(-1, 1), class = "Intercept"),
#         prior(normal(0, 0.5), class = "b"),
#         prior(normal(0, 0.5), class = "sd")
#     ),
#     sample_prior = "only",
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     backend = "rstan",
#     cores = parallel::detectCores(logical=FALSE)
# )
# saveRDS(m6_priors_GabRat, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m6_priors_GabRat.rds")



In [ ]:
# Load models from R
m6_priors_GabRat <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m6_priors_GabRat.rds")

# Extract results
prior_samples_m6_GabRat <- as_draws_df(m6_priors_GabRat)

In [ ]:
# List of datasets and labels
prior_samples <- list(
    GabRat = prior_samples_m6_GabRat
)

# Custom labels for predictors
custom_labels_priors <- c(
    "b_SexF" = "Females",
    "b_MicrohabitatRed_Algae" = "Red Algae",
    "b_MicrohabitatHydroid_Bryozoa" = "Hydroids with Bryozoans",
    "b_Viewpoint10" = "Viewpoint 10",
    "b_Viewpoint20" = "Viewpoint 20"
)

# Custom labels for intercept
custom_labels_priors_intercept <- c(
    "b_Intercept" = "Intercept"
)



# Generate predictor plots
prior_plots <- lapply(
    names(prior_samples),
    function(label) {
        generate_posterior_plot(
            prior_samples[[label]],
            regex_pars = c(
              "b_SexF",
              "b_MicrohabitatRed_Algae",
              "b_MicrohabitatHydroid_Bryozoa",
              "b_Viewpoint10",
              "b_Viewpoint20"
              ),
            x_range = c(-5, 5),
            custom_labels = custom_labels_priors,
            axis_title_y = label %in% c("GabRat")
        )
    }
)
names(prior_plots) <- names(prior_samples)

# Generate intercept plots
prior_plots_intercept <- lapply(
    names(prior_samples),
    function(label) {
        generate_posterior_plot(
            prior_samples[[label]],
            regex_pars = c(
              "b_Intercept"
              ),
            x_range = c(-5, 5),
            custom_labels = custom_labels_priors_intercept,
            axis_title_y = label %in% c("GabRat")
        )
    }
)
names(prior_plots_intercept) <- names(prior_samples)





# Add grey bars with labels for each plot
prior_plots_with_bars <- mapply(
    function(plot, label) {
        patchwork::wrap_elements(create_top_bar(label)) / plot +
            patchwork::plot_layout(heights = c(0.2, 1))
    },
    prior_plots,
    names(prior_plots),
    SIMPLIFY = FALSE
)

prior_plots_intercept_with_bars <- mapply(
    function(plot, label) {
        patchwork::wrap_elements(create_top_bar(label)) / plot +
            patchwork::plot_layout(heights = c(0.2, 1))
    },
    prior_plots_intercept,
    names(prior_plots_intercept),
    SIMPLIFY = FALSE
)




# Combine predictor plots into a 2x3 grid
plot_priors_m6 <- patchwork::wrap_plots(
    prior_plots_with_bars[c("GabRat")],
    ncol = 1
) +
    patchwork::plot_layout(guides = "collect")

# Combine intercept plots into a 2x3 grid
plot_priors_intercept_m6 <- patchwork::wrap_plots(
    prior_plots_intercept_with_bars[c("GabRat")],
    ncol = 1
) +
    patchwork::plot_layout(guides = "collect")




# Add a unified x-axis label for predictor plots
plot_priors_m6 <- plot_priors_m6 +
    patchwork::plot_annotation(
        caption = "Expected value of the odds response (logit scale)",
        theme = theme(plot.caption = element_text(hjust = 0.65, size = 10))
    )

# Add a unified x-axis label for intercept plots
plot_priors_intercept_m6 <- plot_priors_intercept_m6 +
    patchwork::plot_annotation(
        caption = "Expected value of the odds response (logit scale)",
        theme = theme(plot.caption = element_text(hjust = 0.5, size = 10))
    )



ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_priors_m6.png", plot = plot_priors_m6, width = 4.5, height = 4.5, units = "in", dpi = 300)
ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_priors_intercept_m6.png", plot = plot_priors_intercept_m6, width = 4.5, height = 4.5, units = "in", dpi = 300)


In [ ]:
# Convert images to base64 (assuming these return base64 data URIs)
plot_priors_m6 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_priors_m6.png")
plot_priors_intercept_m6 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_priors_intercept_m6.png")

# Create the HTML (vertical display)
html_priors <- paste0("
  <style>
    .image-container {
      display: flex;
      justify-content: space-between;
      align-items: flex-start;
      flex-wrap: wrap;
      gap: 20px;
      max-width: 100%;
    }

    .image-container img {
      flex: 1 1 48%;
      max-width: 48%;
      height: auto;
      border: 1px solid #ccc;
    }

    @media screen and (max-width: 800px) {
      .image-container img {
        max-width: 100%;
        flex: 1 1 100%;
      }
    }
  </style>

  <div class='image-container'>
    <img src='", plot_priors_m6, "' alt='Prior Plot'>
    <img src='", plot_priors_intercept_m6, "' alt='Prior Intercept Plot'>
  </div>
")

# Display the HTML
IRdisplay::display_html(html_priors)




---

### 6. Run final models

Now that we have finalized the model parameters, we fit models using the actual data and compare them to null models to assess the significance of predictors.


>**Note:** We will run these models in RStudio to be consistent because the rstan package sometimes does not like Jupyter. The models were saved in RStudio and loaded below. We left the code chunks as comments for reference.

In [ ]:

# # Set seed for reproducibility
# set.seed(5678)
# # null model (for comparison)
# m6v0_GabRat <- brm(
#     bf(GabRat ~ 1 + (1 | Image)),
#     data = data_m6_clean,
#     family = Beta(),
#     prior = c(
#         prior(normal(-1, 1), class = "Intercept"),
#         prior(normal(0, 0.5), class = "sd")
#     ),
#     sample_prior = TRUE,
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     backend = "rstan",
#     cores = parallel::detectCores(logical=FALSE)
# )
# saveRDS(m6v0_GabRat, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m6v0_GabRat.rds")


# # Set seed for reproducibility
# set.seed(5678)
# # GabRat Model (Sex & Viewpoint & Microhabitat)
# m6v1_GabRat <- brm(
#     bf(GabRat ~ 1 + Viewpoint + Sex + Microhabitat + (1 | Image)),
#     data = data_m6_clean,
#     family = Beta(),
#     prior = c(
#         prior(normal(-1, 1), class = "Intercept"),
#         prior(normal(0, 0.5), class = "b"),
#         prior(normal(0, 0.5), class = "sd")
#     ),
#     sample_prior = TRUE,
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     backend = "rstan",
#     cores = parallel::detectCores(logical=FALSE)
# )
# saveRDS(m6v1_GabRat, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m6v1_GabRat.rds")


# # Set seed for reproducibility
# set.seed(5678)
# # GabRat Model (Sex & Viewpoint & Microhabitat Association)
# m6v2_GabRat <- brm(
#     bf(GabRat ~ 1 + Viewpoint + Sex + Microhabitat_Association + (1 | Image)),
#     data = data_m6_clean,
#     family = Beta(),
#     prior = c(
#         prior(normal(-1, 1), class = "Intercept"),
#         prior(normal(0, 0.5), class = "b"),
#         prior(normal(0, 0.5), class = "sd")
#     ),
#     sample_prior = TRUE,
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     backend = "rstan",
#     cores = parallel::detectCores(logical=FALSE)
# )
# saveRDS(m6v2_GabRat, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m6v2_GabRat.rds")


#### **Model comparison**

Make sure final models outperform the null model, and check to see which model performs best. Use LOO to compare models.

>**Note:** We will added loo to these models in RStudio. The models were saved in RStudio and loaded below. We left the code chunks as comments for reference.

In [ ]:
# # Add LOO to models
# m6v0_GabRat <- add_criterion(m6v0_GabRat, "loo", moment_match = TRUE)
# m6v1_GabRat <- add_criterion(m6v1_GabRat, "loo", moment_match = TRUE)
# m6v2_GabRat <- add_criterion(m6v2_GabRat, "loo", moment_match = TRUE)

# Save models with loo so you don't have to do this again
# saveRDS(m6v0_GabRat, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m6v0_GabRat.rds")
# saveRDS(m6v1_GabRat, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m6v1_GabRat.rds")
# saveRDS(m6v2_GabRat, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m6v2_GabRat.rds")

In [ ]:
# Load models from R
m6v0_GabRat <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m6v0_GabRat.rds")
m6v1_GabRat <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m6v1_GabRat.rds")
m6v2_GabRat <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m6v2_GabRat.rds")


In [ ]:

# Compare all models
loo1_m6v1 <- loo_compare(m6v0_GabRat, m6v1_GabRat)


In [ ]:
# Convert to dataframe
df_loo1_m6v1 <- as.data.frame(loo1_m6v1) %>%
  rownames_to_column(var = "Model")

  # Save loo as tables
write.table(df_loo1_m6v1, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_loo_m6v1.csv", sep = ",", row.names = TRUE, col.names = TRUE)

In [ ]:
# Convert each data frame to a plain HTML table string
table_1_loo_m6v1 <- minimal_html_table(df_loo1_m6v1, caption = "LOO values m6v1 - Edge Disruption")

my_tabs_loo_m6v1 <- '
<style>
/* Basic container styling */
.tabs-container {
  width: 100%;
  margin: 1em 0;
}

/* Hide the radio inputs (we only show their labels as tabs) */
.tabs-container input[type="radio"] {
  display: none;
}

/* The “tab-label” styling: looks like a tab */
.tab-label {
  display: inline-block;
  padding: 10px;
  margin-right: 2px;
  background: #eee;
  border: 1px solid #ccc;
  cursor: pointer;
  border-bottom: none;
}

/* The active tab label */
.tab-label-active {
  background: #fff;
}

/* The panel that holds table content */
.tab-content {
  border: 1px solid #ccc;
  padding: 10px;
  display: none;
}

/* For each radio input, show its corresponding content when checked */
#tab1_loo_m6v1:checked ~ #content1_loo_m6v1 {
  display: block;
}

/* Also style the label of the checked radio as “active” using the :checked + label technique */
#tab1_loo_m6v1:checked + label[for="tab1_loo_m6v1"] {
  background: #fff;
  border-bottom: none;
}
</style>

<div class="tabs-container">

  <!-- 1) Tab radio + label -->
  <input type="radio" name="tabs_loo_m6v1" id="tab1_loo_m6v1" checked>
  <label class="tab-label" for="tab1_loo_m6v1">Table 1</label>

  <!-- Content for each tab -->
  <div class="tab-content" id="content1_loo_m6v1">REPLACE_WITH_table_1_m6v1</div>
</div>
'

# Now do the replacements for each table
my_tabs_loo_m6v1 <- gsub("REPLACE_WITH_table_1_m6v1", table_1_loo_m6v1, my_tabs_loo_m6v1)

IRdisplay::display_html(my_tabs_loo_m6v1)

In [ ]:

# Compare all models
loo1_m6v2 <- loo_compare(m6v0_GabRat, m6v2_GabRat)


In [ ]:

# Convert to dataframe
df_loo1_m6v2 <- as.data.frame(loo1_m6v2) %>%
  rownames_to_column(var = "Model")

  # Save loo as tables
write.table(df_loo1_m6v2, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_loo_m6v2.csv", sep = ",", row.names = TRUE, col.names = TRUE)


In [ ]:

# Convert each data frame to a plain HTML table string
table_1_loo_m6v2 <- minimal_html_table(df_loo1_m6v2, caption = "LOO values m6v2 - Edge Disruption")

my_tabs_loo_m6v2 <- '
<style>
/* Basic container styling */
.tabs-container {
  width: 100%;
  margin: 1em 0;
}

/* Hide the radio inputs (we only show their labels as tabs) */
.tabs-container input[type="radio"] {
  display: none;
}

/* The “tab-label” styling: looks like a tab */
.tab-label {
  display: inline-block;
  padding: 10px;
  margin-right: 2px;
  background: #eee;
  border: 1px solid #ccc;
  cursor: pointer;
  border-bottom: none;
}

/* The active tab label */
.tab-label-active {
  background: #fff;
}

/* The panel that holds table content */
.tab-content {
  border: 1px solid #ccc;
  padding: 10px;
  display: none;
}

/* For each radio input, show its corresponding content when checked */
#tab1_loo_m6v2:checked ~ #content1_loo_m6v2 {
  display: block;
}

/* Also style the label of the checked radio as “active” using the :checked + label technique */
#tab1_loo_m6v2:checked + label[for="tab1_loo_m6v2"] {
  background: #fff;
  border-bottom: none;
}
</style>

<div class="tabs-container">

  <!-- 1) Tab radio + label -->
  <input type="radio" name="tabs_loo_m6v2" id="tab1_loo_m6v2" checked>
  <label class="tab-label" for="tab1_loo_m6v2">Table 1</label>

  <!-- Content for each tab -->
  <div class="tab-content" id="content1_loo_m6v2">REPLACE_WITH_table_1_m6v2</div>
</div>
'

# Now do the replacements for each table
my_tabs_loo_m6v2 <- gsub("REPLACE_WITH_table_1_m6v2", table_1_loo_m6v2, my_tabs_loo_m6v2)

IRdisplay::display_html(my_tabs_loo_m6v2)


#### **Visualize posteriors**

We extract the final model results and visualize the posteriors of our model parameters to get an idea of the significance of the results. This is what we did above when we were evaluating our priors.


In [ ]:

# Extract results
posterior_samples_m6v1_GabRat <- as_draws_df(m6v1_GabRat)
posterior_samples_m6v2_GabRat <- as_draws_df(m6v2_GabRat)


In [ ]:

# List of datasets and labels
posterior_samples <- list(
    GabRat = posterior_samples_m6v1_GabRat
)


# Baseline category data
baseline_data <- tibble(
  parameter = c("Males", "Hydroids", "No correction"),
  mean = 0,  # Centered at 0
  ci_low = -0.2,  # Example CI range
  ci_high = 0.2
)

# Order categories so "Red Algae" appears at the bottom
parameter_order <- c(
  "Males",
  "b_SexF",
  "Hydroids",
  "b_MicrohabitatRed_Algae",
  "b_MicrohabitatHydroid_Bryozoa",
  "No correction",
  "b_Viewpoint10",
  "b_Viewpoint20"
)

#Custom labels
custom_labels_posteriors <- c(
  "Males" = "Males",
  "b_SexF" = "Females",
  "Hydroids" = "Hydroids",
  "b_MicrohabitatRed_Algae" = "Red_Algae",
  "b_MicrohabitatHydroid_Bryozoa" = "Hydroids with Bryozoa",
  "No correction" = "No correction",
  "b_Viewpoint10" = "Viewpoint 10",
  "b_Viewpoint20" = "Viewpoint 20"
)


# Convert the y-axis parameters to factors for consistent alignment
baseline_data$parameter <- factor(baseline_data$parameter, levels = parameter_order)



# Generate plots for predictors
posterior_plots <- lapply(
    names(posterior_samples),
    function(label) {
        generate_posterior_plot(
            posterior_samples[[label]],
            regex_pars = c(
                "b_SexF",
                "b_MicrohabitatRed_Algae",
                "b_MicrohabitatHydroid_Bryozoa",
                "b_Viewpoint10",
                "b_Viewpoint20"
            ),
            x_range = c(-0.5, 0.5),
            custom_labels = custom_labels_posteriors,
            axis_title_y = label %in% c("GabRat")
        ) +
        geom_point(
            data = baseline_data,
            aes(x = mean, y = parameter),
            inherit.aes = FALSE,
            color = "dodgerblue4",
            size = 2
        )
    }
)
names(posterior_plots) <- names(posterior_samples)



# Add grey bars with labels for each plot
posterior_plots_with_bars <- mapply(
    function(plot, label) {
        patchwork::wrap_elements(create_top_bar(label)) / plot +
            patchwork::plot_layout(heights = c(0.2, 1)) # Proportionally adjust grey bar and plot
    },
    posterior_plots,
    names(posterior_plots),
    SIMPLIFY = FALSE
)

# Combine predictor plots into a 2x3 grid with additional spacer
plot_posteriors_m6v1 <- patchwork::wrap_plots(
    patchwork::wrap_plots(
        posterior_plots_with_bars[c("GabRat")],
        ncol = 1
    ))

# Add a unified x-axis label for predictor plots
plot_posteriors_m6v1 <- plot_posteriors_m6v1 +
    patchwork::plot_annotation(
        caption = "Expected value of the response (log scale)",
        theme = theme(plot.caption = element_text(hjust = 0.65, size = 10))
    )

ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_posteriors_m6v1.png", plot = plot_posteriors_m6v1, width = 4.5, height = 4.5, units = "in", dpi = 300)


In [ ]:

# Convert images to base64
plot_posteriors_m6v1 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_posteriors_m6v1.png")

# Create the HTML (vertical display)
html_posteriors_m6v1 <- paste0("
  <style>
    .image-container {
      display: flex;
      justify-content: space-between;
      align-items: flex-start;
      flex-wrap: wrap;
      gap: 20px;
      max-width: 100%;
    }

    .image-container img {
      flex: 1 1 48%;
      max-width: 48%;
      height: auto;
      border: 1px solid #ccc;
    }

    @media screen and (max-width: 800px) {
      .image-container img {
        max-width: 100%;
        flex: 1 1 100%;
      }
    }
  </style>

  <div class='image-container'>
    <img src='", plot_posteriors_m6v1, "' alt='Posterior Plot m1'>
  </div>
")

# Display the HTML
IRdisplay::display_html(html_posteriors_m6v1)

In [ ]:

# List of datasets and labels
posterior_samples <- list(
    GabRat = posterior_samples_m6v2_GabRat
)


# Baseline category data
baseline_data <- tibble(
  parameter = c("Males", "Microhabitat_Association Not Present", "Viewpoint 0"),
  mean = 0,  # Centered at 0
  ci_low = -0.2,  # Example CI range
  ci_high = 0.2
)

# Order categories so "Red Algae" appears at the bottom
parameter_order <- c(
  "Males",
  "b_SexF",
  "Microhabitat_Association Not Present",
  "b_Microhabitat_AssociationPresent",
  "Viewpoint 0",
  "b_Viewpoint10",
  "b_Viewpoint20"
)

#Custom labels
custom_labels_posteriors <- c(
  "Males" = "Males",
  "b_SexF" = "Females",
  "Microhabitat_Association Not Present" = "Unassociated",
  "b_Microhabitat_AssociationPresent" = "Associated",
  "Viewpoint 0" = "Viewpoint 0",
  "b_Viewpoint10" = "Viewpoint 10",
  "b_Viewpoint20" = "Viewpoint 20"
)


# Convert the y-axis parameters to factors for consistent alignment
baseline_data$parameter <- factor(baseline_data$parameter, levels = parameter_order)



# Generate plots for predictors
posterior_plots <- lapply(
    names(posterior_samples),
    function(label) {
        generate_posterior_plot(
            posterior_samples[[label]],
            regex_pars = c(
                "b_SexF",
                "b_Microhabitat_AssociationPresent",
                "b_Viewpoint10",
                "b_Viewpoint20"
            ),
            x_range = c(-0.5, 0.5),
            custom_labels = custom_labels_posteriors,
            axis_title_y = label %in% c("GabRat")
        ) +
        geom_point(
            data = baseline_data,
            aes(x = mean, y = parameter),
            inherit.aes = FALSE,
            color = "dodgerblue4",
            size = 2
        )
    }
)
names(posterior_plots) <- names(posterior_samples)



# Add grey bars with labels for each plot
posterior_plots_with_bars <- mapply(
    function(plot, label) {
        patchwork::wrap_elements(create_top_bar(label)) / plot +
            patchwork::plot_layout(heights = c(0.2, 1)) # Proportionally adjust grey bar and plot
    },
    posterior_plots,
    names(posterior_plots),
    SIMPLIFY = FALSE
)

# Combine predictor plots into a 2x3 grid with additional spacer
plot_posteriors_m6v2 <- patchwork::wrap_plots(
    patchwork::wrap_plots(
        posterior_plots_with_bars[c("GabRat")],
        ncol = 1
    ))

# Add a unified x-axis label for predictor plots
plot_posteriors_m6v2 <- plot_posteriors_m6v2 +
    patchwork::plot_annotation(
        caption = "Expected value of the response (log scale)",
        theme = theme(plot.caption = element_text(hjust = 0.65, size = 10))
    )

ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_posteriors_m6v2.png", plot = plot_posteriors_m6v2, width = 4.5, height = 4.5, units = "in", dpi = 300)


In [ ]:

# Convert images to base64
plot_posteriors_m6v2 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_posteriors_m6v2.png")

# Create the HTML (vertical display)
html_posteriors_m6v2 <- paste0("
  <style>
    .image-container {
      display: flex;
      justify-content: space-between;
      align-items: flex-start;
      flex-wrap: wrap;
      gap: 20px;
      max-width: 100%;
    }

    .image-container img {
      flex: 1 1 48%;
      max-width: 48%;
      height: auto;
      border: 1px solid #ccc;
    }

    @media screen and (max-width: 800px) {
      .image-container img {
        max-width: 100%;
        flex: 1 1 100%;
      }
    }
  </style>

  <div class='image-container'>
    <img src='", plot_posteriors_m6v2, "' alt='Posterior Plot m1'>
  </div>
")

# Display the HTML
IRdisplay::display_html(html_posteriors_m6v2)



The graph of the posteriors gives us an idea of the significance of each predictor. We need to follow up with an evaluation of the model performance before we can trust these results.

---

### 7. Evaluate model performance

#### **Trace plots**

Visualize parameter sampling across iterations to confirm convergence. Each chain should wander around the same mean value without any strong upward or downward trends. "fuzzy caterpillar" or "horizontal band." 

In [ ]:
# Generate trace plots for all models
trace_plots_m6v1 <- generate_trace_plot(
  model = m6v1_GabRat,
  regex_pars = c("b_SexF", "b_MicrohabitatRed_Algae", "b_MicrohabitatHydroid_Bryozoa", "b_Viewpoint10", "b_Viewpoint20"),
  plot_title = "Edge Disruption Model",
  axis_title_y = TRUE,
  axis_text_x = TRUE,
  axis_title_size = 10,
  axis_text_size = 8
)

ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/trace_plots_m6v1.png", plot = trace_plots_m6v1, width = 6, height = 3, units = "in", dpi = 300)


In [ ]:
# Convert images to base64 (if not already done)
trace_plots_m6v1 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/trace_plots_m6v1.png")


# Create the HTML (vertical display)
html_trace_plots_m6v1 <- paste0("
<style>
  .grid-container {
    display: grid;
    grid-template-columns: repeat(1, 1fr); /* 1 columns per row */
    gap: 3px;
    padding: 3px;
    justify-items: center;
  }

  .grid-container img {
    max-width: 600px;
    width: 100%;
    height: auto;
    border: 1px solid #ccc;
  }
</style>

<div class='grid-container'>
  <img src='", trace_plots_m6v1, "' alt='Trace Plot m1'>
</div>
")

# Display the HTML
IRdisplay::display_html(html_trace_plots_m6v1)

In [ ]:
# Generate trace plots for all models
trace_plots_m6v2 <- generate_trace_plot(
  model = m6v2_GabRat,
  regex_pars = c("b_SexF", "b_Microhabitat_AssociationPresent", "b_Viewpoint10", "b_Viewpoint20"),
  plot_title = "Edge Disruption Model",
  axis_title_y = TRUE,
  axis_text_x = TRUE,
  axis_title_size = 10,
  axis_text_size = 8
)

ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/trace_plots_m6v2.png", plot = trace_plots_m6v2, width = 6, height = 3, units = "in", dpi = 300)

# Convert images to base64 (if not already done)
trace_plots_m6v2 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/trace_plots_m6v2.png")



In [ ]:

# Create the HTML (vertical display)
html_trace_plots_m6v2 <- paste0("
<style>
  .grid-container {
    display: grid;
    grid-template-columns: repeat(1, 1fr); /* 1 columns per row */
    gap: 3px;
    padding: 3px;
    justify-items: center;
  }

  .grid-container img {
    max-width: 600px;
    width: 100%;
    height: auto;
    border: 1px solid #ccc;
  }
</style>

<div class='grid-container'>
  <img src='", trace_plots_m6v2, "' alt='Trace Plot m2'>
</div>
")

# Display the HTML
IRdisplay::display_html(html_trace_plots_m6v2)



#### **Posterior predictive checks**

Simulate data based on the models and compare to observed data to verify goodness-of-fit. We do this using "pp_check". The observed data (black line/dots) should sit comfortably within the distribution of simulated data (colored areas or lines)


In [ ]:
# Generate posterior predictive check plots for all models
pp_check_plots_m6v1 <-  generate_pp_check(
  model = m6v1_GabRat,
  nreps = 100,
  axis_title_y = TRUE,
  y_label = "Density",
  plot_title = "Edge Disruption Model",
  axis_title_size = 10,  # custom title size
  axis_text_size = 8    # custom tick label size
)


ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/pp_check_plots_m6v1.png", plot = pp_check_plots_m6v1, width = 3, height = 3, units = "in", dpi = 300)


In [ ]:
# Convert images to base64
pp_check_plots_m6v1 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/pp_check_plots_m6v1.png")


# Create the HTML (horizontal display)
html_pp_check_plots_m6v1 <- paste0("
  <style>
    .image-container {
      display: flex;
      justify-content: space-between;
      align-items: flex-start;
      flex-wrap: wrap;
      gap: 20px;
      max-width: 100%;
    }

    .image-container img {
      flex: 1 1 48%;
      max-width: 48%;
      height: auto;
      border: 1px solid #ccc;
    }

    @media screen and (max-width: 800px) {
      .image-container img {
        max-width: 100%;
        flex: 1 1 100%;
      }
    }
  </style>

  <div class='image-container'>
    <img src='", pp_check_plots_m6v1, "' alt='PP Plot m1'>
  </div>
")

# Display the HTML
IRdisplay::display_html(html_pp_check_plots_m6v1)

In [ ]:
# Generate posterior predictive check plots for all models
pp_check_plots_m6v2 <-  generate_pp_check(
  model = m6v2_GabRat,
  nreps = 100,
  axis_title_y = TRUE,
  y_label = "Density",
  plot_title = "Edge Disruption Model",
  axis_title_size = 10,  # custom title size
  axis_text_size = 8    # custom tick label size
)


ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/pp_check_plots_m6v2.png", plot = pp_check_plots_m6v2, width = 3, height = 3, units = "in", dpi = 300)


In [ ]:

# Convert images to base64
pp_check_plots_m6v2 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/pp_check_plots_m6v2.png")


# Create the HTML (horizontal display)
html_pp_check_plots_m6v2 <- paste0("
  <style>
    .image-container {
      display: flex;
      justify-content: space-between;
      align-items: flex-start;
      flex-wrap: wrap;
      gap: 20px;
      max-width: 100%;
    }

    .image-container img {
      flex: 1 1 48%;
      max-width: 48%;
      height: auto;
      border: 1px solid #ccc;
    }

    @media screen and (max-width: 800px) {
      .image-container img {
        max-width: 100%;
        flex: 1 1 100%;
      }
    }
  </style>

  <div class='image-container'>
    <img src='", pp_check_plots_m6v2, "' alt='PP Plot m2'>
  </div>
")

# Display the HTML
IRdisplay::display_html(html_pp_check_plots_m6v2)



#### **Check convergence**

Check that all $\hat{R}$ values are close to 1, indicating good convergence.


In [ ]:
# Create a helper function
extract_rhat <- function(model, model_name) {
  rhat(model) %>%
    as.data.frame() %>%
    rownames_to_column(var = "Parameter") %>%
    dplyr::filter(startsWith(Parameter, "b_")) %>%   # <-- keep only b_ terms
    dplyr::rename(Rhat = 2) %>%
    dplyr::mutate(Model = model_name) %>%
    dplyr::mutate(across(where(is.numeric), ~ signif(.x, digits = 3)))
}

# Extract for each model group
rhat_m6v1 <- extract_rhat(m6v1_GabRat, "GabRat")
rhat_m6v2 <- extract_rhat(m6v2_GabRat, "GabRat")


In [ ]:
# Save tables
write.table(rhat_m6v1, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_rhat_m6v1.csv", sep = ",", row.names = FALSE, col.names = TRUE)
write.table(rhat_m6v2, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_rhat_m6v2.csv", sep = ",", row.names = FALSE, col.names = TRUE)

In [ ]:
# Convert each data frame to a plain HTML table string
table_1_rhat <- minimal_html_table(rhat_m6v1, caption = "Rhat values - Edge Disruption")
table_2_rhat <- minimal_html_table(rhat_m6v2, caption = "Rhat values - Edge Disruption")


my_tabs_rhat <- '
<style>
/* Basic container styling */
.tabs-container {
  width: 100%;
  margin: 1em 0;
}

/* Hide the radio inputs (we only show their labels as tabs) */
.tabs-container input[type="radio"] {
  display: none;
}

/* The “tab-label” styling: looks like a tab */
.tab-label {
  display: inline-block;
  padding: 10px;
  margin-right: 2px;
  background: #eee;
  border: 1px solid #ccc;
  cursor: pointer;
  border-bottom: none;
}

/* The active tab label */
.tab-label-active {
  background: #fff;
}

/* The panel that holds table content */
.tab-content {
  border: 1px solid #ccc;
  padding: 10px;
  display: none;
}

/* For each radio input, show its corresponding content when checked */
#tab1_rhat:checked ~ #content1_rhat,
#tab2_rhat:checked ~ #content2_rhat {
  display: block;
}

/* Also style the label of the checked radio as “active” using the :checked + label technique */
#tab1_rhat:checked + label[for="tab1_rhat"],
#tab2_rhat:checked + label[for="tab2_rhat"] {
  background: #fff;
  border-bottom: none;
}
</style>

<div class="tabs-container">

  <!-- 1) Tab radio + label -->
  <input type="radio" name="tabs_rhat" id="tab1_rhat" checked>
  <label class="tab-label" for="tab1_rhat">Table 1</label>

  <!-- 2) Tab radio + label -->
  <input type="radio" name="tabs_rhat" id="tab2_rhat">
  <label class="tab-label" for="tab2_rhat">Table 2</label>

  <!-- Content for each tab -->
  <div class="tab-content" id="content1_rhat">REPLACE_WITH_table_1</div>
  <div class="tab-content" id="content2_rhat">REPLACE_WITH_table_2</div>
</div>
'

# Replace both placeholders
my_tabs_rhat <- gsub("REPLACE_WITH_table_1", table_1_rhat, my_tabs_rhat)
my_tabs_rhat <- gsub("REPLACE_WITH_table_2", table_2_rhat, my_tabs_rhat)

# Display HTML
IRdisplay::display_html(my_tabs_rhat)


#### **Check uncertainty**

Extract parameter estimates and their confidence intervals to assess the significance of the predictors on color pattern metrics. We check 85% and 95% confidence intervals. Summaries are displayed in tables for all models. 



In [ ]:
# Extract summaries for each variable
extract_summary <- function(model, prob_85, prob_95) {
    summary_85 <- summary(model, prob = prob_85)
    summary_95 <- summary(model, prob = prob_95)

    as.data.frame(summary_85$fixed) %>%
        dplyr::select("Estimate", "Est.Error", "l-85% CI", "u-85% CI") %>%
        mutate(
            "l-95% CI" = summary_95$fixed$`l-95% CI`,
            "u-95% CI" = summary_95$fixed$`u-95% CI`
        ) %>%
        mutate(across(where(is.numeric), ~ signif(.x, digits = 3))) %>%
        rownames_to_column(var = "Parameter") # Add rownames as Parameter column
}

uncertainty1 <- extract_summary(m6v1_GabRat, prob_85 = 0.85, prob_95 = 0.95)
uncertainty2 <- extract_summary(m6v2_GabRat, prob_85 = 0.85, prob_95 = 0.95)


In [ ]:

# Save tables
write.table(uncertainty1, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_uncertainty_m6v1.csv", sep = ",", row.names = TRUE, col.names = TRUE)
write.table(uncertainty2, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_uncertainty_m6v2.csv", sep = ",", row.names = TRUE, col.names = TRUE)

In [ ]:
# Convert each data frame to a plain HTML table string
table_1_uncertainty <- minimal_html_table(uncertainty1, caption = "Uncertainty values - Edge Disruption")
table_2_uncertainty <- minimal_html_table(uncertainty2, caption = "Uncertainty values - Edge Disruption")

my_tabs_uncertainty <- '
<style>
/* Basic container styling */
.tabs-container {
  width: 100%;
  margin: 1em 0;
}

/* Hide the radio inputs (we only show their labels as tabs) */
.tabs-container input[type="radio"] {
  display: none;
}

/* The “tab-label” styling: looks like a tab */
.tab-label {
  display: inline-block;
  padding: 10px;
  margin-right: 2px;
  background: #eee;
  border: 1px solid #ccc;
  cursor: pointer;
  border-bottom: none;
}

/* The active tab label */
.tab-label-active {
  background: #fff;
}

/* The panel that holds table content */
.tab-content {
  border: 1px solid #ccc;
  padding: 10px;
  display: none;
}

/* For each radio input, show its corresponding content when checked */
#tab1_uncertainty:checked ~ #content1_uncertainty,
#tab2_uncertainty:checked ~ #content2_uncertainty {
  display: block;
}

/* Also style the label of the checked radio as “active” using the :checked + label technique */
#tab1_uncertainty:checked + label[for="tab1_uncertainty"],
#tab2_uncertainty:checked + label[for="tab2_uncertainty"] {
  background: #fff;
  border-bottom: none;
}
</style>

<div class="tabs-container">

  <!-- 1) Tab radio + label -->
  <input type="radio" name="tabs_uncertainty" id="tab1_uncertainty" checked>
  <label class="tab-label" for="tab1_uncertainty">Table 1</label>

  <!-- 2) Tab radio + label -->
  <input type="radio" name="tabs_uncertainty" id="tab2_uncertainty">
  <label class="tab-label" for="tab2_uncertainty">Table 2</label>

  <!-- Content for each tab -->
  <div class="tab-content" id="content1_uncertainty">REPLACE_WITH_table_1</div>
  <div class="tab-content" id="content2_uncertainty">REPLACE_WITH_table_2</div>
</div>
'

# Replace both placeholders
my_tabs_uncertainty <- gsub("REPLACE_WITH_table_1", table_1_uncertainty, my_tabs_uncertainty)
my_tabs_uncertainty <- gsub("REPLACE_WITH_table_2", table_2_uncertainty, my_tabs_uncertainty)

# Display HTML
IRdisplay::display_html(my_tabs_uncertainty)



---

#### **Check posterior probabilities**

Now we can evaluate our hypotheses using the posterior distributions of the model parameters. Remember, we are concerned with two things:
1. Is there a difference in edge disruption between females and males? (Sex effect)
2. Is there a difference in edge disruption between microhabitats? (Microhabitat effect)

We will calculate the posterior probability of these effects.

In [ ]:
##### **Posterior Probability of Sex Effect**

# Extract posterior draws
draws_m6v1 <- as_draws_df(m6v1_GabRat)
draws_m6v2 <- as_draws_df(m6v2_GabRat)

# Compute posterior probabilities for each model
pp_m6v1_positive <- mean(draws_m6v1$b_SexF > 0)
pp_m6v1_negative <- mean(draws_m6v1$b_SexF < 0)

pp_m6v2_positive <- mean(draws_m6v2$b_SexF > 0)
pp_m6v2_negative <- mean(draws_m6v2$b_SexF < 0)

# Create summary tables
pp_summary_m6v1_sex <- data.frame(
  Hypothesis = c("P(Sex effect GabRat > 0)", "P(Sex effect GabRat < 0)"),
  PosteriorProbability = c(pp_m6v1_positive, pp_m6v1_negative)
)

pp_summary_m6v2_sex <- data.frame(
  Hypothesis = c("P(Sex effect GabRat > 0)", "P(Sex effect GabRat < 0)"),
  PosteriorProbability = c(pp_m6v2_positive, pp_m6v2_negative)
)

# Save combined table
pp_summary_sex_combined <- rbind(pp_summary_m6v1_sex, pp_summary_m6v2_sex)
write.table(pp_summary_sex_combined, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_postprob_m6_sex.csv", sep = ",", row.names = TRUE, col.names = TRUE)


In [ ]:

# Convert each to HTML
html_m6v1_sex <- minimal_html_table(pp_summary_m6v1_sex, caption = "Posterior probabilities (Sex) - GabRat")
html_m6v2_sex <- minimal_html_table(pp_summary_m6v2_sex, caption = "Posterior probabilities (Sex) - GabRat")

# Tabs layout
my_tabs_pp_sex <- '
<style>
.tabs-container {
  width: 100%;
  margin: 1em 0;
}
.tabs-container input[type="radio"] {
  display: none;
}
.tab-label {
  display: inline-block;
  padding: 10px;
  margin-right: 2px;
  background: #eee;
  border: 1px solid #ccc;
  cursor: pointer;
  border-bottom: none;
}
.tab-content {
  border: 1px solid #ccc;
  padding: 10px;
  display: none;
}
#tab_m6v1_sex:checked ~ #content_m6v1_sex,
#tab_m6v2_sex:checked ~ #content_m6v2_sex {
  display: block;
}
#tab_m6v1_sex:checked + label[for="tab_m6v1_sex"],
#tab_m6v2_sex:checked + label[for="tab_m6v2_sex"] {
  background: #fff;
  border-bottom: none;
}
</style>

<div class="tabs-container">
  <input type="radio" name="tabs_pp_sex" id="tab_m6v1_sex" checked>
  <label class="tab-label" for="tab_m6v1_sex">Microhabitat Model</label>

  <input type="radio" name="tabs_pp_sex" id="tab_m6v2_sex">
  <label class="tab-label" for="tab_m6v2_sex">Microhabitat_Association Model</label>

  <div class="tab-content" id="content_m6v1_sex">REPLACE_WITH_m6v1_sex</div>
  <div class="tab-content" id="content_m6v2_sex">REPLACE_WITH_m6v2_sex</div>
</div>
'

# Inject tables
my_tabs_pp_sex <- gsub("REPLACE_WITH_m6v1_sex", html_m6v1_sex, my_tabs_pp_sex)
my_tabs_pp_sex <- gsub("REPLACE_WITH_m6v2_sex", html_m6v2_sex, my_tabs_pp_sex)

# Display
IRdisplay::display_html(my_tabs_pp_sex)


In [ ]:
##### **Posterior Probability of Microhabitat Effect**

# Extract posterior draws
draws_m6v1 <- as_draws_df(m6v1_GabRat)
draws_m6v2 <- as_draws_df(m6v2_GabRat)

# Compute posterior probabilities for each model
pp_RedAlgae_m6v1_positive <- mean(draws_m6v1$b_MicrohabitatRed_Algae > 0)
pp_HydroidBryozoa_m6v1_positive <- mean(draws_m6v1$b_MicrohabitatHydroid_Bryozoa > 0)
pp_RedAlgae_m6v1_negative <- mean(draws_m6v1$b_MicrohabitatRed_Algae < 0)
pp_HydroidBryozoa_m6v1_negative <- mean(draws_m6v1$b_MicrohabitatHydroid_Bryozoa < 0)

pp_m6v2_positive <- mean(draws_m6v2$b_Microhabitat_AssociationPresent > 0)
pp_m6v2_negative <- mean(draws_m6v2$b_Microhabitat_AssociationPresent < 0)

# Create summary tables
pp_summary_m6v1_microhabitat <- data.frame(
  Hypothesis = c(
    "P(Microhabitat Red Algae effect GabRat > 0)",
    "P(Microhabitat Hydroid with Bryozoa effect GabRat > 0)",
    "P(Microhabitat Red Algae effect GabRat < 0)",
    "P(Microhabitat Hydroid with Bryozoa effect GabRat < 0)"
    ),
  PosteriorProbability = c(
    pp_RedAlgae_m6v1_positive, 
    pp_HydroidBryozoa_m6v1_positive,
    pp_RedAlgae_m6v1_negative,
    pp_HydroidBryozoa_m6v1_negative
    )
)

pp_summary_m6v2_microhabitat <- data.frame(
  Hypothesis = c(
    "P(Microhabitat Association effect GabRat > 0)",
    "P(Microhabitat Association effect GabRat < 0)"
    ),
  PosteriorProbability = c(
    pp_m6v2_positive, 
    pp_m6v2_negative
    )
)

# Save combined table
pp_summary_microhabitat_combined <- rbind(pp_summary_m6v1_microhabitat, pp_summary_m6v2_microhabitat)
write.table(pp_summary_microhabitat_combined, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_postprob_m6_microhabitat.csv", sep = ",", row.names = TRUE, col.names = TRUE)


In [ ]:
# Convert each to HTML
html_m6v1_microhabitat <- minimal_html_table(pp_summary_m6v1_microhabitat, caption = "Posterior probabilities (Microhabitat) - GabRat")
html_m6v2_microhabitat <- minimal_html_table(pp_summary_m6v2_microhabitat, caption = "Posterior probabilities (Microhabitat) - GabRat")

# Tabs layout
my_tabs_pp_microhabitat <- '
<style>
.tabs-container {
  width: 100%;
  margin: 1em 0;
}
.tabs-container input[type="radio"] {
  display: none;
}
.tab-label {
  display: inline-block;
  padding: 10px;
  margin-right: 2px;
  background: #eee;
  border: 1px solid #ccc;
  cursor: pointer;
  border-bottom: none;
}
.tab-content {
  border: 1px solid #ccc;
  padding: 10px;
  display: none;
}
#tab_m6v1_microhabitat:checked ~ #content_m6v1_microhabitat,
#tab_m6v2_microhabitat:checked ~ #content_m6v2_microhabitat {
  display: block;
}
#tab_m6v1_microhabitat:checked + label[for="tab_m6v1_microhabitat"],
#tab_m6v2_microhabitat:checked + label[for="tab_m6v2_microhabitat"] {
  background: #fff;
  border-bottom: none;
}
</style>

<div class="tabs-container">
  <input type="radio" name="tabs_pp_microhabitat" id="tab_m6v1_microhabitat" checked>
  <label class="tab-label" for="tab_m6v1_microhabitat">Microhabitat Model</label>

  <input type="radio" name="tabs_pp_microhabitat" id="tab_m6v2_microhabitat">
  <label class="tab-label" for="tab_m6v2_microhabitat">Microhabitat_Association Model</label>

  <div class="tab-content" id="content_m6v1_microhabitat">REPLACE_WITH_m6v1_microhabitat</div>
  <div class="tab-content" id="content_m6v2_microhabitat">REPLACE_WITH_m6v2_microhabitat</div>
</div>
'

# Inject tables
my_tabs_pp_microhabitat <- gsub("REPLACE_WITH_m6v1_microhabitat", html_m6v1_microhabitat, my_tabs_pp_microhabitat)
my_tabs_pp_microhabitat <- gsub("REPLACE_WITH_m6v2_microhabitat", html_m6v2_microhabitat, my_tabs_pp_microhabitat)

# Display
IRdisplay::display_html(my_tabs_pp_microhabitat)

In [ ]:
##### **Posterior Probability of Viewpoint Effect**

# Extract posterior draws
draws_m6v1 <- as_draws_df(m6v1_GabRat)
draws_m6v2 <- as_draws_df(m6v2_GabRat)

# Compute posterior probabilities for each model
pp_viewpoint10_m6v1_positive <- mean(draws_m6v1$b_Viewpoint10 > 0)
pp_viewpoint10_m6v1_negative <- mean(draws_m6v1$b_Viewpoint10 < 0)
pp_viewpoint20_m6v1_positive <- mean(draws_m6v1$b_Viewpoint20 > 0)
pp_viewpoint20_m6v1_negative <- mean(draws_m6v1$b_Viewpoint20 < 0)

pp_viewpoint10_m6v2_positive <- mean(draws_m6v2$b_Viewpoint10 > 0)
pp_viewpoint10_m6v2_negative <- mean(draws_m6v2$b_Viewpoint10 < 0)
pp_viewpoint20_m6v2_positive <- mean(draws_m6v2$b_Viewpoint20 > 0)
pp_viewpoint20_m6v2_negative <- mean(draws_m6v2$b_Viewpoint20 < 0)

# Create summary tables
pp_summary_m6v1_viewpoint <- data.frame(
  Hypothesis = c(
    "P(10 mm acuity effect GabRat > 0)",
    "P(20 mm acuity effect GabRat > 0)",
    "P(10 mm acuity effect GabRat < 0)",
    "P(20 mm acuity effect GabRat < 0)"
    ),
  PosteriorProbability = c(
    pp_viewpoint10_m6v1_positive,
    pp_viewpoint20_m6v1_positive,
    pp_viewpoint20_m6v1_negative,
    pp_viewpoint20_m6v1_negative
    )
)

pp_summary_m6v2_viewpoint <- data.frame(
  Hypothesis = c(
    "P(10 mm acuity effect GabRat > 0)",
    "P(20 mm acuity effect GabRat > 0)",
    "P(10 mm acuity effect GabRat < 0)",
    "P(20 mm acuity effect GabRat < 0)"
    ),
  PosteriorProbability = c(
    pp_viewpoint10_m6v2_positive,
    pp_viewpoint20_m6v2_positive,
    pp_viewpoint20_m6v2_negative,
    pp_viewpoint20_m6v2_negative
    )
)

# Save combined table
pp_summary_viewpoint_combined <- rbind(pp_summary_m6v1_viewpoint, pp_summary_m6v2_viewpoint)
write.table(pp_summary_viewpoint_combined, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_postprob_m6_viewpoint.csv", sep = ",", row.names = TRUE, col.names = TRUE)


In [ ]:

# Convert each to HTML
html_m6v1_viewpoint <- minimal_html_table(pp_summary_m6v1_viewpoint, caption = "Posterior probabilities (Viewpoint) - GabRat")
html_m6v2_viewpoint <- minimal_html_table(pp_summary_m6v2_viewpoint, caption = "Posterior probabilities (Viewpoint) - GabRat")

# Tabs layout
my_tabs_pp_viewpoint <- '
<style>
.tabs-container {
  width: 100%;
  margin: 1em 0;
}
.tabs-container input[type="radio"] {
  display: none;
}
.tab-label {
  display: inline-block;
  padding: 10px;
  margin-right: 2px;
  background: #eee;
  border: 1px solid #ccc;
  cursor: pointer;
  border-bottom: none;
}
.tab-content {
  border: 1px solid #ccc;
  padding: 10px;
  display: none;
}
#tab_m6v1_viewpoint:checked ~ #content_m6v1_viewpoint,
#tab_m6v2_viewpoint:checked ~ #content_m6v2_viewpoint {
  display: block;
}
#tab_m6v1_viewpoint:checked + label[for="tab_m6v1_viewpoint"],
#tab_m6v2_viewpoint:checked + label[for="tab_m6v2_viewpoint"] {
  background: #fff;
  border-bottom: none;
}
</style>

<div class="tabs-container">
  <input type="radio" name="tabs_pp_viewpoint" id="tab_m6v1_viewpoint" checked>
  <label class="tab-label" for="tab_m6v1_viewpoint">Microhabitat Model</label>

  <input type="radio" name="tabs_pp_viewpoint" id="tab_m6v2_viewpoint">
  <label class="tab-label" for="tab_m6v2_viewpoint">Microhabitat_Association Model</label>

  <div class="tab-content" id="content_m6v1_viewpoint">REPLACE_WITH_m6v1_viewpoint</div>
  <div class="tab-content" id="content_m6v2_viewpoint">REPLACE_WITH_m6v2_viewpoint</div>
</div>
'

# Inject tables
my_tabs_pp_viewpoint <- gsub("REPLACE_WITH_m6v1_viewpoint", html_m6v1_viewpoint, my_tabs_pp_viewpoint)
my_tabs_pp_viewpoint <- gsub("REPLACE_WITH_m6v2_viewpoint", html_m6v2_viewpoint, my_tabs_pp_viewpoint)

# Display
IRdisplay::display_html(my_tabs_pp_viewpoint)


---

### 8. Summarize results

The predictor Sex does not have a significant effect on edge disruption (i.e., GabRat values) at 85% and 95% confidence intervals.


#### **Make final figures**

Generate tidy figures for the Results section of our report/paper.


In [ ]:
##### FINAL PLOT FOR WNAN PAPER (MODEL 6 ONLY)

# Define predictors_list for GabRat
predictors_list_gabrat <- list(
  list(
    name = "Sex",
    model = "m2",
    baseline = tibble(parameter = "Male", mean = 0),
    order = c("Male", "b_SexF"),
    labels = c("Male" = "Male", "b_SexF" = "Female"),
    regex = "b_SexF"
  ),
  list(
    name = "Microhabitat",
    model = "m1",
    baseline = tibble(parameter = "Hydroids", mean = 0),
    order = c("Hydroids", "b_MicrohabitatRed_Algae", "b_MicrohabitatHydroid_Bryozoa"),
    labels = c(
      "Hydroids" = "Hydroids",
      "b_MicrohabitatRed_Algae" = "Red Algae",
      "b_MicrohabitatHydroid_Bryozoa" = "Bryozoa"
    ),
    regex = "b_Microhabitat"
  ),
  list(
    name = "Microhabitat_Association",
    model = "m2",
    baseline = tibble(parameter = "Microhabitat_Association Not Present", mean = 0),
    order = c("Microhabitat_Association Not Present", "b_Microhabitat_AssociationPresent"),
    labels = c(
      "Microhabitat_Association Not Present" = "Unassociated",
      "b_Microhabitat_AssociationPresent" = "Associated"
    ),
    regex = "b_Microhabitat_AssociationPresent"
  ),
  list(
    name = "Viewpoint",
    model = "m2",
    baseline = tibble(parameter = "No correction", mean = 0),
    order = c("No correction", "b_Viewpoint10", "b_Viewpoint20"),
    labels = c(
      "No correction" = "No correction",
      "b_Viewpoint10" = "10 mm acuity",
      "b_Viewpoint20" = "20 mm acuity"
    ),
    regex = "b_Viewpoint"
  )
)

# GabRat posterior samples for each model
posterior_samples_all_gabrat <- list(
  m1 = list(GabRat = posterior_samples_m6v1_GabRat),
  m2 = list(GabRat = posterior_samples_m6v2_GabRat)
)

# Updated extract_effects() for GabRat
extract_effects_gabrat <- function(posterior_df, predictor_cfg, predictor_name = "GabRat") {
  matched_cols <- posterior_df %>%
    dplyr::select(matches(predictor_cfg$regex)) %>%
    colnames()

  if (length(matched_cols) == 0) {
    stop(paste("No columns matched regex:", predictor_cfg$regex, "for predictor:", predictor_cfg$name))
  }

  draws <- posterior_df %>%
    dplyr::select(all_of(matched_cols)) %>%
    pivot_longer(cols = everything(), names_to = "parameter", values_to = "value") %>%
    mutate(
      label = predictor_cfg$labels[parameter],
      predictor = predictor_name,
      group = predictor_cfg$name
    )

  baseline <- predictor_cfg$baseline %>%
    mutate(
      label = predictor_cfg$labels[parameter],
      predictor = predictor_name,
      group = predictor_cfg$name,
      value = mean
    )

  bind_rows(draws, baseline)
}

# Build combined plot data for GabRat
plot_data_gabrat <- map_dfr(
  predictors_list_gabrat,
  function(predictor_cfg) {
    model_id <- predictor_cfg$model
    extract_effects_gabrat(
      posterior_df = posterior_samples_all_gabrat[[model_id]]$GabRat,
      predictor_cfg = predictor_cfg
    )
  }
)

# Order y-axis
plot_data_gabrat$label <- factor(plot_data_gabrat$label, levels = c(
  "Male", "Female",
  "Hydroids", "Red Algae", "Bryozoa",
  "Unassociated", "Associated",
  "No correction", "10 mm acuity", "20 mm acuity"
))


# Final plot
final_plot_posteriors_m6 <- ggplot(plot_data_gabrat, aes(x = value, y = label, fill = group, color = group)) +
  stat_halfeye(
    .width = c(0.85, 0.95),
    slab_alpha = 0.4,
    interval_size_range = c(0.75, 0),
    normalize = "groups",
    slab_linewidth = 0.6,  # now visible
    point_size = 1.25
  ) +
  geom_point(
    data = filter(plot_data_gabrat, !is.na(mean)),
    aes(x = mean),
    inherit.aes = TRUE,
    size = 0.75,
    shape = 21
  ) +
  geom_vline(xintercept = 0, size = 0.6, linetype = 3, color = "red") +
  facet_wrap(~ predictor, ncol = 7, scales = "free_x", labeller = label_parsed) +
  scale_fill_manual(
    values = c(
      "Sex" = "purple",
      "Microhabitat" = "forestgreen",
      "Microhabitat_Association" = "#ff9500",
      "Viewpoint" = "steelblue"
    )
  ) +
  scale_color_manual(
    values = c(
      "Sex" = "purple",
      "Microhabitat" = "forestgreen",
      "Microhabitat_Association" = "#ff9500",
      "Viewpoint" = "steelblue"
    )
  ) +
  labs(
    x = "Effect size (log scale)",
    y = NULL
  ) +
  theme_bw(base_size = 8) +
  theme(
    strip.text = element_text(face = "bold", size = 10),
    axis.text.y = element_text(size = 8),
    panel.spacing = unit(0.5, "lines"),
    panel.grid.major.x = element_blank(),
    panel.grid.minor.x = element_blank(),
    legend.position = "none"
  )

# Save it
ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/final_plot_posteriors_m6.png", plot = final_plot_posteriors_m6, width = 4.5, height = 5, units = "in", dpi = 300)


In [ ]:
# Convert images to base64
final_plot_posteriors_m6 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/final_plot_posteriors_m6.png")

# Create the HTML 
html_posteriors_viewpoint_WNAN_final <- paste0("
  <style>
    .image-row {
      display: flex;
      gap: 20px;
      justify-content: center;
      align-items: flex-start;
    }
    .image-row img {
      max-width: 100%;
      height: auto;
      border: 1px solid #ccc;
    }
  </style>
<div class='image-row'>
  <img src='", final_plot_posteriors_m6, "' alt='Final Microhabitat Posterior Plot'>
</div>
")

# Display the HTML
IRdisplay::display_html(html_posteriors_viewpoint_WNAN_final)

In [ ]:
##### FINAL PLOT FOR WNAN PAPER (COMBINED WITH MODEL 5)

# Load models from R
m5v1_e_max_diff <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m5v1_e_max_diff.rds")
m5v1_Filter_max_diff <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m5v1_Filter_max_diff.rds")
m5v1_e_prop_diff <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m5v1_e_prop_diff.rds")
m5v1_R_c_diff <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m5v1_R_c_diff.rds")
m5v1_G_c_diff <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m5v1_G_c_diff.rds")
m5v1_B_c_diff <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m5v1_B_c_diff.rds")

m5v2_e_max_diff <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m5v2_e_max_diff.rds")
m5v2_Filter_max_diff <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m5v2_Filter_max_diff.rds")
m5v2_e_prop_diff <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m5v2_e_prop_diff.rds")
m5v2_R_c_diff <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m5v2_R_c_diff.rds")
m5v2_G_c_diff <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m5v2_G_c_diff.rds")
m5v2_B_c_diff <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m5v2_B_c_diff.rds")

m6v1_GabRat <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m6v1_GabRat.rds")
m6v2_GabRat <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m6v2_GabRat.rds")

# Extract results
posterior_samples_m5v1_e_max_diff <- as_draws_df(m5v1_e_max_diff)
posterior_samples_m5v1_Filter_max_diff <- as_draws_df(m5v1_Filter_max_diff)
posterior_samples_m5v1_e_prop_diff <- as_draws_df(m5v1_e_prop_diff)
posterior_samples_m5v1_R_c_diff <- as_draws_df(m5v1_R_c_diff)
posterior_samples_m5v1_G_c_diff <- as_draws_df(m5v1_G_c_diff)
posterior_samples_m5v1_B_c_diff <- as_draws_df(m5v1_B_c_diff)

posterior_samples_m5v2_e_max_diff <- as_draws_df(m5v2_e_max_diff)
posterior_samples_m5v2_Filter_max_diff <- as_draws_df(m5v2_Filter_max_diff)
posterior_samples_m5v2_e_prop_diff <- as_draws_df(m5v2_e_prop_diff)
posterior_samples_m5v2_R_c_diff <- as_draws_df(m5v2_R_c_diff)
posterior_samples_m5v2_G_c_diff <- as_draws_df(m5v2_G_c_diff)
posterior_samples_m5v2_B_c_diff <- as_draws_df(m5v2_B_c_diff)

posterior_samples_m6v1_GabRat <- as_draws_df(m6v1_GabRat)
posterior_samples_m6v2_GabRat <- as_draws_df(m6v2_GabRat)


# Define predictors list for Q3m5 (6 predictors)
predictors_list <- list(
  list(
    name = "Sex",
    model = "m1",
    baseline = tibble(parameter = "Male", mean = 0),
    order = c("Male", "b_SexF"),
    labels = c("Male" = "Male", "b_SexF" = "Female"),
    regex = "b_SexF"
  ),
  list(
    name = "Microhabitat",
    model = "m1",
    baseline = tibble(parameter = "Hydroids", mean = 0),
    order = c("Hydroids", "b_MicrohabitatRed_Algae", "b_MicrohabitatHydroid_Bryozoa"),
    labels = c(
      "Hydroids" = "Hydroids",
      "b_MicrohabitatRed_Algae" = "Red Algae",
      "b_MicrohabitatHydroid_Bryozoa" = "Bryozoa"
    ),
    regex = "b_Microhabitat"
  ),
  list(
    name = "Microhabitat_Association",
    model = "m2",
    baseline = tibble(parameter = "Microhabitat_Association Not Present", mean = 0),
    order = c("Microhabitat_Association Not Present", "b_Microhabitat_AssociationPresent"),
    labels = c(
      "Microhabitat_Association Not Present" = "Unassociated",
      "b_Microhabitat_AssociationPresent" = "Associated"
    ),
    regex = "b_Microhabitat_AssociationPresent"
  ),
  list(
    name = "Viewpoint",
    model = "m1",
    baseline = tibble(parameter = "No correction", mean = 0),
    order = c("No correction", "b_Viewpoint10", "b_Viewpoint20"),
    labels = c(
      "No correction" = "No correction",
      "b_Viewpoint10" = "10 mm acuity",
      "b_Viewpoint20" = "20 mm acuity"
    ),
    regex = "b_Viewpoint"
  )
)

# Posterior samples by model and variable (Q3m5 predictors)
posterior_samples_all <- list(
  m1 = list(
    `e[max]` = posterior_samples_m5v1_e_max_diff,
    `Filter[max]` = posterior_samples_m5v1_Filter_max_diff,
    `e[prop]` = posterior_samples_m5v1_e_prop_diff,
    `R[c]` = posterior_samples_m5v1_R_c_diff,
    `G[c]` = posterior_samples_m5v1_G_c_diff,
    `B[c]` = posterior_samples_m5v1_B_c_diff
  ),
  m2 = list(
    `e[max]` = posterior_samples_m5v2_e_max_diff,
    `Filter[max]` = posterior_samples_m5v2_Filter_max_diff,
    `e[prop]` = posterior_samples_m5v2_e_prop_diff,
    `R[c]` = posterior_samples_m5v2_R_c_diff,
    `G[c]` = posterior_samples_m5v2_G_c_diff,
    `B[c]` = posterior_samples_m5v2_B_c_diff
  )
)

# Posterior samples for GabRat
posterior_samples_all_gabrat <- list(
  m1 = list(GabRat = posterior_samples_m6v1_GabRat),
  m2 = list(GabRat = posterior_samples_m6v2_GabRat)
)

# GabRat predictors
predictors_list_gabrat <- list(
  list(
    name = "Sex",
    model = "m2",
    baseline = tibble(parameter = "Male", mean = 0),
    order = c("Male", "b_SexF"),
    labels = c("Male" = "Male", "b_SexF" = "Female"),
    regex = "b_SexF"
  ),
  list(
    name = "Microhabitat",
    model = "m1",
    baseline = tibble(parameter = "Hydroids", mean = 0),
    order = c("Hydroids", "b_MicrohabitatRed_Algae", "b_MicrohabitatHydroid_Bryozoa"),
    labels = c(
      "Hydroids" = "Hydroids",
      "b_MicrohabitatRed_Algae" = "Red Algae",
      "b_MicrohabitatHydroid_Bryozoa" = "Bryozoa"
    ),
    regex = "b_Microhabitat"
  ),
  list(
    name = "Microhabitat_Association",
    model = "m2",
    baseline = tibble(parameter = "Microhabitat_Association Not Present", mean = 0),
    order = c("Microhabitat_Association Not Present", "b_Microhabitat_AssociationPresent"),
    labels = c(
      "Microhabitat_Association Not Present" = "Unassociated",
      "b_Microhabitat_AssociationPresent" = "Associated"
    ),
    regex = "b_Microhabitat_AssociationPresent"
  ),
  list(
    name = "Viewpoint",
    model = "m2",
    baseline = tibble(parameter = "No correction", mean = 0),
    order = c("No correction", "b_Viewpoint10", "b_Viewpoint20"),
    labels = c(
      "No correction" = "No correction",
      "b_Viewpoint10" = "10 mm acuity",
      "b_Viewpoint20" = "20 mm acuity"
    ),
    regex = "b_Viewpoint"
  )
)

# Unified extraction function
extract_effects <- function(posterior_df, predictor_cfg, predictor_name) {
  matched_cols <- posterior_df %>%
    dplyr::select(matches(predictor_cfg$regex)) %>%
    colnames()

  if (length(matched_cols) == 0) {
    stop(paste("No columns matched regex:", predictor_cfg$regex, "for predictor:", predictor_cfg$name))
  }

  draws <- posterior_df %>%
    dplyr::select(all_of(matched_cols)) %>%
    pivot_longer(cols = everything(), names_to = "parameter", values_to = "value") %>%
    mutate(
      label = predictor_cfg$labels[parameter],
      predictor = predictor_name,
      group = predictor_cfg$name
    )

  baseline <- predictor_cfg$baseline %>%
    mutate(
      label = predictor_cfg$labels[parameter],
      predictor = predictor_name,
      group = predictor_cfg$name,
      value = mean
    )

  bind_rows(draws, baseline)
}

# Build Q3m5 predictor plot_data
plot_data <- map_dfr(
  names(posterior_samples_all$m1),
  function(pred_name) {
    map_dfr(
      predictors_list,
      function(predictor_cfg) {
        model_id <- predictor_cfg$model
        extract_effects(
          posterior_df = posterior_samples_all[[model_id]][[pred_name]],
          predictor_cfg = predictor_cfg,
          predictor_name = pred_name
        )
      }
    )
  }
)

# Build GabRat plot_data
plot_data_gabrat <- map_dfr(
  predictors_list_gabrat,
  function(predictor_cfg) {
    model_id <- predictor_cfg$model
    extract_effects(
      posterior_df = posterior_samples_all_gabrat[[model_id]]$GabRat,
      predictor_cfg = predictor_cfg,
      predictor_name = "GabRat"
    )
  }
)

# Add predictor label to GabRat
plot_data_gabrat$predictor <- "GabRat"

# Combine and factor
label_order <- c(
  "Male", "Female",
  "Hydroids", "Red Algae", "Bryozoa",
  "Unassociated", "Associated",
  "No correction", "10 mm acuity", "20 mm acuity"
)
plot_data$label <- factor(plot_data$label, levels = label_order)
plot_data_gabrat$label <- factor(plot_data_gabrat$label, levels =label_order)
plot_data_combined <- bind_rows(plot_data, plot_data_gabrat)

# Set predictor column order
plot_data_combined$predictor <- factor(
  plot_data_combined$predictor,
  levels = c("e[max]", "Filter[max]", "e[prop]", "R[c]", "G[c]", "B[c]", "GabRat"),
  labels = c(
    expression(Delta*e[max]),
    expression(Delta*Filter[max]),
    expression(Delta*e[prop]),
    expression(Delta*R[c]),
    expression(Delta*G[c]),
    expression(Delta*B[c]),
    expression(GabRat)
  )
)

# Final plot
final_plot_posteriors_m5_m6 <- ggplot(plot_data_combined, aes(x = value, y = label, fill = group, color = group)) +
  stat_halfeye(
    .width = c(0.85, 0.95),
    slab_alpha = 0.4,
    interval_size_range = c(0.75, 0),
    normalize = "groups",
    slab_linewidth = 0.6,  # now visible
    point_size = 1.25
  ) +
  geom_point(
    data = filter(plot_data_combined, !is.na(mean)),
    aes(x = mean),
    inherit.aes = TRUE,
    size = 0.75,
    shape = 21
  ) +
  geom_vline(xintercept = 0, size = 0.6, linetype = 3, color = "red") +
  facet_wrap(~ predictor, ncol = 7, scales = "free_x", labeller = label_parsed) +
  scale_fill_manual(
    values = c(
      "Sex" = "purple",
      "Microhabitat" = "forestgreen",
      "Microhabitat_Association" = "#ff9500",
      "Viewpoint" = "steelblue"
    )
  ) +
  scale_color_manual(
    values = c(
      "Sex" = "purple",
      "Microhabitat" = "forestgreen",
      "Microhabitat_Association" = "#ff9500",
      "Viewpoint" = "steelblue"
    )
  ) +
  labs(
    x = "Effect size",
    y = NULL
  ) +
  theme_bw(base_size = 8) +
  theme(
    strip.text = element_text(face = "bold", size = 10),
    axis.text.y = element_text(size = 8),
    panel.spacing = unit(0.5, "lines"),
    panel.grid.major.x = element_blank(),
    panel.grid.minor.x = element_blank(),
    legend.position = "none"
  )




# Save plot
ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/final_plot_posteriors_m5_m6.png", plot = final_plot_posteriors_m5_m6, width = 9, height = 4.5, units = "in", dpi = 300)



In [ ]:

# Convert images to base64
final_plot_posteriors_m5_m6 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/final_plot_posteriors_m5_m6.png")

# Create the HTML 
html_m5_m6_posteriors_WNAN_final <- paste0("
  <style>
    .image-row {
      display: flex;
      gap: 20px;
      justify-content: center;
      align-items: flex-start;
    }
    .image-row img {
      max-width: 100%;
      height: auto;
      border: 1px solid #ccc;
    }
  </style>
<div class='image-row'>
  <img src='", final_plot_posteriors_m5_m6, "' alt='Final Microhabitat Posterior Plot'>
</div>
")

# Display the HTML
IRdisplay::display_html(html_m5_m6_posteriors_WNAN_final)